## A Numerical Solver for the Compact Domains Problem

### Review of the Problem

This Jupyter notebook accompanies Section 4.5 of my PhD Thesis, and provides a method to explicitly compute the dual supremum $\mathcal{S}_{X,Y}(\mu,m)$ and the primal infimum $\mathcal{I}_{X,Y}(\mu,m)$ where $Y=[0,R]^2$. It also allows us to compute the dual optimizer $(\varphi^{\lambda_0},\lambda_0).$ Stability issues mean that, in theory, this algorithm may not always identify a primal optimizer $\gamma_0,$ but the algorithm becomes more robust if we replace each point of $\mu$ with a small point cloud.  

First, import some libraries:

In [1]:
import numpy as np
import scipy.optimize as opt
import warnings
warnings.filterwarnings("ignore", category=np.ComplexWarning)

Next, we implement Equation (4.6) in Proposition 4.13 of my thesis which, given $x\in [0,\infty)\times (0,\infty),$ $\lambda\in \R^2$, and $R>0$, allows us to compute $$Y(x;\lambda)\in \operatorname{argmin}_{y\in[0,R]^2} [c(x,y)-\varphi(x)-\lambda\cdot y].$$
For the purpose of simplifying the numerics, we assume, in the case where  $\operatorname{argmin}_{y\in[0,R]^2} [c(x,y)-\varphi(x)-\lambda\cdot y]$ is a line segment, $Y(x;\lambda)$ has maximum possible norm (i.e. either $Y_1(x;\lambda)=R$ or $Y_2(x;\lambda)=R$). This may lead to overestimating the primal cost, and violation of the mean constraint, but such issues seem rare in practice, and are likely to have only minor impacts for measures whose support has large cardinality. 

Additionally, the fact that $\operatorname{argmin}_{y\in[0,R]^2} [c(x,y)-\varphi(x)-\lambda\cdot y]$ is not a continuous function of $x$ may lead to numerical errors when solving the primal problem. As such, it is important to check that the resulting optimal primal measure $\gamma_0$ indeed satisfies the mean constraint, and to compare $\int c\; d\gamma_0$ with the dual supremum. 

In [2]:
# Start by defining the auxiliary functions from Proposition 4.13
def func_Lda1(x1 : float, x2: float, lda1 : float) -> float:
    return  x1 +  x2 * lda1 
def func_Lda2(x1 : float, x2: float, lda2 : float) -> float:
    return x1**2 - 2 * x2 * lda2 

# Next, define the cost function from Equation (1.5)
def func_c(x1: float, x2: float, y1: float, y2: float) -> float:
    if (x2 > 0) and (y2 > 0):
        return (y2/(2*x2))*(x1-y1/y2)**2
    elif x1 * y2 - y1 == 0:
        return 0
    else: 
        return float('inf') 
    
# Compute $Y$ as in Equation (4.6) of my thesis.
def func_Y(x1 : float, x2: float, lda1 : float, lda2 : float, R : float) -> np.ndarray:
    Lda1 = func_Lda1(x1, x2, lda1)
    Lda2 = func_Lda2(x1, x2, lda2)
    if (Lda1 <= 0) and (Lda2 <= 0):
        return np.array([0,R])
    elif (0 <= Lda1 <=1) and (Lda2 <= Lda1**2):
        return R * np.array([Lda1,1])
    elif (Lda2 <= 1 <= Lda1):
        return np.array([R,R])
    elif (Lda1 >= 1) and (1 <= Lda2 <= Lda1**2):
        return R*np.array([1, 1/np.sqrt(Lda2)])
    else:
        return np.array([0,0])
    

Now, we define a measure and compute key quantities. We encode discrete probability measures as `numpy` `ndarrays`, where each row has three entries -- the mass, the $x_1$ coordinate, and the $x_2$ coordinate. We also set a mean target and a variance target.

In [3]:
# Define one sample measure  with a small number of points in its support
mu_denorm= np.array([[1, 1, 1], [1, 2, 1], [1, 1, 2], [1, 2,2]], dtype = 'f')

# Automatically normalize
mu_norm = mu_denorm
mu_norm[:,0] = mu_norm[:,0]/(np.sum(mu_norm[:,0], axis = 0))

# Set mean target
mean = np.array([1.3,1.5])
# Set size of $Y$
side_length = 3

The choice of $Y=[0,R]^2$ along with the presence of a prescribed mean, allows us to compute $\varphi^\lambda$ (which is the optimal choice of $\varphi$ admissible for the dual supremum) as in Equation (4.7). This function is continuous in $x$ and $\lambda$, so should not cause issues with numerical computation of the dual supremum. 


In [4]:
def phi_lambda(x1: float, x2: float, lda1: float, lda2: float, R : float) -> float:
    Lda1 = func_Lda1(x1, x2, lda1)
    Lda2 = func_Lda2(x1, x2, lda2)
    if (Lda1 <= 0) and (Lda2 <= 0):
        return (R * Lda2) / (2 * x2)
    elif (0 <= Lda1 <=1) and (Lda2 <= Lda1**2):
        return R*(Lda2 - Lda1**2)/(2 * x2)
    elif (Lda2 <= 1 <= Lda1):
        return R*(Lda2 - 2 * Lda1 + 1)/(2 * x2)
    elif (Lda1 >= 1) and (1 <= Lda2 <= Lda1**2):
        return R*(np.sqrt(Lda2) -  Lda1)/(x2)
    else:
        return 0


Next, we define a function to compute the dual objective function 
$$\int \varphi\; d\mu+\lambda\cdot y.$$

In [5]:
def objective_function(params, R, m, measure) -> float:
    lda1, lda2 = params
    # negative because scipy can only minimize
    return (-sum([measure[i,0] * phi_lambda(measure[i, 1], measure[i, 2], lda1, lda2, R) for i in range(len(measure))])
            - lda1 * m[0] - lda2 * m[1])

We will also wish to compute the mean $$\int Y(\cdot;\lambda)\; d\mu$$ to verify if our result actually achieves the mean constraint:

In [6]:
def mu_mean(params, R, measure) -> np.ndarray:
    lda1, lda2 = params
    return sum([measure[i,0] * func_Y(measure[i,1], measure[i,2], lda1, lda2, R) for i in range(len(measure))])

We can also recover the $(\Lambda_1,\Lambda_2)$ for each point in the support of a given $\mu$; this is useful for debugging.

In [7]:
def Ldas(measure, lda):
    return {tuple(measure[i,1:]) : 
            [func_Lda1(measure[i,1],measure[i,2], lda[0]),func_Lda2(measure[i,1],measure[i,2], lda[1])] 
            for i in range(len(measure))}

Moreover, we compute $$\int c(x, Y(x;\lambda))d\mu(x)$$ in order to get a sense of the value of the primal infimum.

In [8]:
def primal_cost(params, R, measure) -> np.ndarray:
    lda1, lda2 = params
    return sum([measure[i,0] * func_c(measure[i,1], measure[i,2], 
                                      func_Y(measure[i,1], measure[i,2], lda1, lda2, R)[0],
                                      func_Y(measure[i,1], measure[i,2], lda1, lda2, R)[1]) 
                                      for i in range(len(measure))])

Finally, for technical reasons, we also create a wrapper so that we can use SciPy's optimize function. 

In [9]:
def make_objective(R, m, measure):
    def wrapped_objective(params):
        return objective_function(params, R, m, measure)
    return wrapped_objective

Next, we finally complete the optimization, using the Nelder-Mead method:

In [10]:
optimization_result = opt.minimize(make_objective(side_length, mean, mu_norm), [-1,1], method = 'Nelder-Mead')
print(optimization_result)

       message: Optimization terminated successfully.
       success: True
        status: 0
           fun: -0.008888888888888946
             x: [-8.889e-02  1.460e-01]
           nit: 69
          nfev: 127
 final_simplex: (array([[-8.889e-02,  1.460e-01],
                       [-8.889e-02,  1.459e-01],
                       [-8.889e-02,  1.460e-01]]), array([-8.889e-03, -8.889e-03, -8.889e-03]))


We recover the optimal choice of $\lambda.$ 

In [11]:
optimal_lda = optimization_result.x
print("lambda = " +  str(optimal_lda))

lambda = [-0.08888889  0.14595935]


Next, we verify that the result achieves the target mean, which is the case for `mu_norm`. We also compute the value of the primal infimum and dual supremum, finding that they are equal in this case. 

In [12]:
print("mean: " +  str(mu_mean(optimal_lda, side_length, mu_norm)))
print("dual value: " + str(-objective_function(optimal_lda, side_length, mean, mu_norm)))
print("primal value: " + str(primal_cost(optimal_lda, side_length,  mu_norm)))

mean: [1.29999999 1.5       ]
dual value: 0.008888888888888946
primal value: 0.008888889787490745


However, changing the mean can lead to errors with the primal infimum, as we see in the following, where $m=(1,2)$ rather than $(1.3, 1.5)$. In particular, we see that the mean constraint is violated, and the primal value dips below the dual value.

In [13]:
optimization_result_new = opt.minimize(make_objective(side_length, np.array([1,2]), mu_norm), [-1,1], method = 'Nelder-Mead')
optimal_lda_new = optimization_result_new.x
print("lambda = " +  str(optimal_lda_new))
print("mean: " +  str(mu_mean(optimal_lda_new, side_length, mu_norm)))
print("dual value: " + str(-objective_function(optimal_lda_new, side_length, np.array([1,2]), mu_norm)))
print("primal value: " + str(primal_cost(optimal_lda_new, side_length,  mu_norm)))

lambda = [-0.46156262  0.71008519]
mean: [0.4614841 1.5      ]
dual value: 0.3461538418133254
primal value: 0.23967006321533613


Upon closer inspection (see below), with the optimal choice of $\lambda_0\approx (-0.4616, 0.7101)$, the point $x_0=(2,2)\in \operatorname{spt}(\mu)$ satisfies 
$$\Lambda_2(x_0;\lambda_0)\approx 1.1597 \approx(1.0769)^2\approx(\Lambda_1(x_0;\lambda_0))^2,$$
meaning that this case is approximately on the threshold between the case $\Lambda_1\ge 1$ and $1\le\Lambda_2<\Lambda_1^2$ and the case $\Lambda_2>0$ and $\Lambda_1<\sqrt{\Lambda_2}$ described in Equation (4.6). In the former case, the mass at $x_0$ would be assigned to $(R, R/\sqrt{\Lambda_2})$, and in the latter, the mass at $x_0$ would be assigned to $(0,0)$, which hints at instability in the numerics. In particular, we suspect that the mass at $x_0$ should be assigned to $(R, R/\sqrt{\Lambda_2})$ (or somewhere on the line segment between this point and the origin), but small numerical errors lead to this mass being assigned to $(0,0).$

In [14]:
Ldas(mu_norm, optimal_lda_new)

{(1.0, 1.0): [0.5384373756559031, -0.42017037142645286],
 (2.0, 1.0): [1.538437375655903, 2.579829628573547],
 (1.0, 2.0): [0.07687475131180632, -1.8403407428529057],
 (2.0, 2.0): [1.0768747513118062, 1.1596592571470943]}

To resolve this, we use the following function to replace $\mu$ with a slightly perturbed version, where each point mass is broken into a cloud of smaller point masses near the original point. 

In [15]:
def make_cloud(mu, K=1024, radius=0.001, rng=None):
    if rng is None:
        rng = np.random.default_rng(12345)

    masses = np.repeat(mu[:, 0] / K, K)
    locations = np.repeat(mu[:, 1:], K, axis=0)

    noise = rng.uniform(-radius, radius, size=locations.shape)
    locations = locations + noise

    return np.column_stack([masses, locations])

We now run the algorithm with the perturbed version, finding that the optimal $\lambda$ and dual supremum are largely unchanged, but now the resulting measure (nearly) attains the mean constraint, and the primal infimum is significantly closer to, but still smaller than, the dual supremum.

In [16]:
mu_cloud = make_cloud(mu_norm)
optimization_result_cloud = opt.minimize(make_objective(side_length, np.array([1,2]), mu_cloud), [-1,1], method = 'Nelder-Mead')
optimal_lda_cloud = optimization_result_cloud.x
print("lambda = " +  str(optimal_lda_cloud))
print("mean: " +  str(mu_mean(optimal_lda_cloud, side_length, mu_cloud)))
print("dual value: " + str(-objective_function(optimal_lda_cloud, side_length, np.array([1,2]), mu_cloud)))
print("primal value: " + str(primal_cost(optimal_lda_cloud, side_length,  mu_cloud)))

lambda = [-0.46143207  0.71013525]
mean: [0.99789762 1.99816643]
dual value: 0.346078563744302
primal value: 0.3457465905032643
